In [1]:
import init
import epsilon_rho
import metrics
import optimize_algorithm as optimize
import numpy as np

In [2]:
def quantum_process_tomography_dephasing (num_qubits, gamma, num_loop, rho, kraus_operators):
    rho2 = epsilon_rho.calculate_dephasing(rho, num_qubits, gamma)
    # Optimizing
    out_kraus_operators = optimize.optimize_adam(rho, rho2, kraus_operators, num_qubits, 0.005, num_loop)

    _cost = 1 - metrics.compilation_trace_fidelity(rho, epsilon_rho.calculate_from_kraus_operators(rho2, out_kraus_operators))
    return rho2, out_kraus_operators, _cost


In [9]:
import os
np.set_printoptions(threshold=np.inf)
def write_result (folder, num_qubits, rho, kraus_operators, g):
        folder_path = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)
        folder_path = os.path.join(folder, folder_path)
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)

        rho2, out_kraus_operators, _cost = quantum_process_tomography_dephasing(num_qubits, g, 700, rho, kraus_operators)
        
        rho2_file_path = os.path.join(folder_path, 'rho2.txt')
        kraus_file_path = os.path.join(folder_path, 'kraus_operators.txt')
        cost_file_path = os.path.join(folder_path, 'cost.txt')
        fidelity_file_path = os.path.join(folder_path, 'fidelity.txt')
        
        print(g, num_qubits, _cost)
        
        rho3 = epsilon_rho.calculate_from_kraus_operators(rho2, out_kraus_operators)
        with open(rho2_file_path, 'w') as file:
            file.write(str(rho2))
        with open(kraus_file_path, 'w') as file:
            file.write(str(out_kraus_operators.numpy()))
        with open(cost_file_path, 'w') as file:
            file.write(str(_cost.numpy().real))
        with open(fidelity_file_path, 'w') as file:
            file.write(str(1 - metrics.compilation_trace_fidelity(rho, rho3).numpy().real))



In [11]:
'''plotting
fix ite, run gamma & num-qubit
fix gamma, fix num-qubit, tính p' và p'2 dagger, 
lưu kraus và cost cuối cùng
'''

experiment_folder = ''
g_shift = 0.05

for num_qubits in range(5, 6):
    if (experiment_folder == ''):
        break
    else:
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)
    rho = init.create_random_state(num_qubits=num_qubits)
    unitary_matrix = init.create_unitary_matrix(num_qubits=num_qubits)
    kraus_operators = init.create_kraus_operators_from_unitary(unitary_matrix=unitary_matrix)
    
    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho))

    g = 0
    while g < 1:
        write_result(experiment_folder, num_qubits, rho, kraus_operators, g)
    
    

In [ ]:
def write_result_2(folder, folder_path, num_qubits, rho, kraus_operators, g): 
    folder_path = os.path.join(folder, folder_path)
    os.makedirs(folder_path, exist_ok=True)  # Create directory if it doesn't exist

    rho2, out_kraus_operators, _cost = quantum_process_tomography_dephasing(num_qubits, g, 700, rho, kraus_operators)
    
    # File paths
    rho2_file_path = os.path.join(folder_path, 'rho2.txt')
    kraus_file_path = os.path.join(folder_path, 'kraus_operators.txt')
    trace_rho2_file_path = os.path.join(folder_path, 'tr_rho2.txt')
    trace_out_rho2_file_path = os.path.join(folder_path, 'tr_out_rho2.txt')
    
    out_rho2 = epsilon_rho.calculate_from_kraus_operators(rho, out_kraus_operators)
    # Define Pauli-X matrix
    pauli_X = np.array([[0, 1], [1, 0]], dtype=np.complex128)
    # Calculate tr(X1, rho)
    trace_rho2 = metrics.trace_Pauli(rho2, 0, pauli_X)
    trace_out_rho2 = metrics.trace_Pauli(out_rho2, 0, pauli_X)
    
    print(g, num_qubits, _cost)
    
    # Save results
    with open(rho2_file_path, 'w') as file:
        file.write(str(rho2))
    with open(kraus_file_path, 'w') as file:
        file.write(str(out_kraus_operators.numpy()))
    with open(trace_rho2_file_path, 'w') as file:
        file.write(str(trace_rho2.real))
    with open(trace_out_rho2_file_path, 'w') as file:
        file.write(str(trace_out_rho2.real))

In [ ]:
'''plotting
fix ite, run gamma & num-qubit
fix gamma, fix num-qubit, tính rho2 và rho2_out, 
lưu kraus, rho2 và rho2_out, Z_a và Z_b đối với mỗi rho2
'''

base_folder = 'experiment_trace_Z'
g_shift = 0.05

for num_qubits in range(1, 3):
    if (base_folder == ''):
        break
    else:
        experiment_folder = os.path.join(base_folder, str(num_qubits))
        if not os.path.exists(experiment_folder):
            os.makedirs(experiment_folder)
    rho = init.create_random_state(num_qubits=num_qubits)
    unitary_matrix = init.create_unitary_matrix(num_qubits=num_qubits)
    kraus_operators = init.create_kraus_operators_from_unitary(unitary_matrix=unitary_matrix)
    
    rho_file_path = os.path.join(experiment_folder, str(num_qubits)+"_qubits_rho.txt")
    with open(rho_file_path, 'w') as file:
        file.write(str(rho))

    g = 0
    t = 0
    while g < 1:
        print(g, num_qubits, t)
        folder_path_a = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)+'_a'
        folder_path_b = str(num_qubits)+"_qubits_"+"{:.3f}".format(g)+'_b'
        write_result_2(experiment_folder, folder_path_a, 
                       num_qubits, rho, kraus_operators, 1 - np.exp(-2 * g * t))
        write_result_2(experiment_folder, folder_path_b,
                       num_qubits, rho, kraus_operators, 1 - np.exp(-2 * g * t**2))
        g = round(g + g_shift, 3)
        t = t+1
        
    